<a href="https://colab.research.google.com/github/YounSooKimTech/self_study/blob/main/ModelSelection_Test_ML_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This is the file which explain why I choose random forest classifier
# So, this is the file for explanation
# if you want to use the model, you can ignore this code file

In [26]:
# Javier made the label for the traiing dataset for L2 risk labeling! Thanks!
# use fillna to prevent possible errros

import pandas as pd

df_train = pd.read_excel('/content/BOS_RISKS (1).xlsx')
df_train.head()

# fill the NaN value as blank since having NaN can cuase a problem
df_train["RISK_DESCRIPTION"].fillna("", inplace=True)
df_train["Standard_risk_L2"].fillna("", inplace=True)

In [27]:
! pip install deep_translator

In [28]:
# translate the text to increase the number of data with labels to feed the model

from deep_translator import GoogleTranslator

def translate_in_english(text):
  translated = GoogleTranslator(source="auto", target="en").translate(text)
  return translated

df_train["translated"] = df_train["RISK_DESCRIPTION"].apply(translate_in_english)

In [29]:
# tokenization for better processing
# remove punctuation and number

import spacy
nlp = spacy.load("en_core_web_sm")

def tokenize_lemma_and_remove_stopwords_and_punctuation(text):
    doc = nlp(text)
    tokens = [token.lemma_.lower() for token in doc if not token.is_punct and not token.is_stop and not token.like_num]
    tokens = [token for token in tokens if token.strip()]
    return " ".join(tokens)

df_train["text"] = df_train["translated"].apply(tokenize_lemma_and_remove_stopwords_and_punctuation)


In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

# for convinience, train and test for X and y
X = df_train['text']
y = df_train['Standard_risk_L2']

# split the dataset to test the models
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state= 42)

# TF-IDF vectorization (trained by X_train)
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # You can adjust the number of features as needed
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)



In [31]:
# Naive Bayes Classifier

from sklearn.naive_bayes import MultinomialNB

nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_tfidf, y_train)
predicted_labels_nb = nb_classifier.predict(X_test_tfidf)

accuracy_nb = accuracy_score(y_test, predicted_labels_nb)

print("Accuracy (Naive Bayes):", accuracy_nb)

Accuracy (Naive Bayes): 0.47058823529411764


In [32]:
# Logistic Regression as a base model
from sklearn.linear_model import LogisticRegression

logistic_regression_classifier = LogisticRegression(random_state=42)
logistic_regression_classifier.fit(X_train_tfidf, y_train)
predicted_labels_lr = logistic_regression_classifier.predict(X_test_tfidf)

accuracy_lr = accuracy_score(y_test, predicted_labels_lr)

print("Accuracy (Logistic Regression):", accuracy_lr)

Accuracy (Logistic Regression): 0.5882352941176471


In [33]:
# random forest classifier
from sklearn.ensemble import RandomForestClassifier

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train_tfidf, y_train)
predicted_labels_rf = rf_classifier.predict(X_test_tfidf)

accuracy_rf = accuracy_score(y_test, predicted_labels_rf)

print("Accuracy (Random Forest):", accuracy_rf)

Accuracy (Random Forest): 0.6078431372549019
